In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = mls
band = [9]
commitnumber = de5d43e
molecule = {'ch4': 1.8e-06}
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o ch4 band7
  atmpro = mls
  band = 7
  commitnumber = a06b618
  conc = 1.8e-06
  dv = 0.001
  klin = 2e-21
  molecule = ch4
  ng_adju = [0, 0]
  ng_refs = [2, 3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 1380
  vmin = 1215
  w_diffuse = [(1.66, 1.66), (1.66, 1.66, 1.66)]
  wgt = [(0.7, 0.7), (0.4, 0.6, 0.75)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0 -0.103256                  NaN       NaN
0.000312      NaN       NaN                  1.0 -0.062979
0.000750      2.0 -0.102576                  2.0 -0.062538
0.001052      3.0 -0.101012                  3.0 -0.061495
0.001476      4.0 -0.099346                  4.0 -0.060400
0.002070      5.0 -0.097681                  5.0 -0.059287
0.002904      6.0 -0.096051                  6.0 -0.058160
0.004074      7.0 -0.094462                  7.0 -0.057011
0.005714      8.0 -0.092941                  8.0 -0.055858
0.008015      9.0 -0.091520                  9.0 -0.054682
0.011243     10.0 -0.090253                 10.0 -0.053516
0.015771     11.0 -0.089215                 11.0 -0.052335
0.022122     12.0 -0.088495                 12.0 -0.051181
0.031031     13.0 -0.088230                 13.0 -0.050045
0.043528     14.0 -0.088501                 14.0 -0.048880
0.061057     15.0 -0.085147                 15.0 -0.044861
0.085645     16.0 -0.069571                 16.0 -0.032203
0.120136     17.0 -0.041543                 17.0 -0.010382
0.168516     18.0 -0.006824                 18.0  0.017319
0.236378     19.0  0.034932                 19.0  0.051543
0.331549     20.0  0.084869                 20.0  0.093475
0.465100     21.0  0.144491                 21.0  0.144365
0.652400     22.0  0.215988                 22.0  0.205183
0.915100     23.0  0.279324                 23.0  0.260255
1.283650     24.0  0.269195                 24.0  0.260366
1.800600     25.0  0.196056                 25.0  0.208193
2.525700     26.0  0.126686                 26.0  0.149481
3.542800     27.0  0.076681                 27.0  0.098260
4.969550     28.0  0.044832                 28.0  0.058212
6.970850     29.0  0.027253                 29.0  0.031312
9.778100     30.0  0.018033                 30.0  0.016015
13.715850    31.0  0.011998                 31.0  0.008475
19.239350    32.0  0.007031                 32.0  0.005019
26.987250    33.0  0.003814                 33.0  0.003917
37.855300    34.0  0.001530                 34.0  0.002707
53.100050    35.0 -0.002111                 35.0 -0.001064
73.887500    36.0 -0.007288                 36.0 -0.007041
97.662500    37.0 -0.011910                 37.0 -0.012630
121.437500   38.0 -0.015828                 38.0 -0.017542
145.212500   39.0 -0.021121                 39.0 -0.023559
168.987500   40.0 -0.026877                 40.0 -0.029574
192.762500   41.0 -0.029327                 41.0 -0.032134
216.537500   42.0 -0.027948                 42.0 -0.030952
240.312500   43.0 -0.026171                 43.0 -0.028998
264.087500   44.0 -0.024502                 44.0 -0.027031
287.862500   45.0 -0.022747                 45.0 -0.024985
311.637500   46.0 -0.020828                 46.0 -0.022781
335.412500   47.0 -0.018711                 47.0 -0.020508
359.187500   48.0 -0.016389                 48.0 -0.018085
382.962500   49.0 -0.013865                 49.0 -0.015496
406.737500   50.0 -0.011169                 50.0 -0.012808
430.512500   51.0 -0.008370                 51.0 -0.010078
454.287500   52.0 -0.005549                 52.0 -0.007360
478.062500   53.0 -0.002756                 53.0 -0.004669
501.837500   54.0  0.000022                 54.0 -0.002014
525.612500   55.0  0.002842                 55.0  0.000675
549.387500   56.0  0.005755                 56.0  0.003480
573.162500   57.0  0.008795                 57.0  0.006423
596.937500   58.0  0.011982                 58.0  0.009528
620.712500   59.0  0.015333                 59.0  0.012815
644.487500   60.0  0.018861                 60.0  0.016336
668.262500   61.0  0.022585                 61.0  0.020100
692.037500   62.0  0.026532                 62.0  0.024111
715.812500   63.0  0.030731                 63.0  0.028398
739.587500   64.0  0.035215                 64.0  0.033006
76

# Fluxes by Layer

CRD                          CLIRAD                \
                         flug      fldg      fnetg       flug          fldg   
pressure     level                                                            
0.000000e+00 1     -19.274965  0.000000 -19.274965        NaN           NaN   
1.000000e-08 1            NaN       NaN        NaN -19.301753  8.944598e-08   
6.244000e-04 2     -19.274975  0.000003 -19.274973 -19.301759  1.848539e-06   
8.759000e-04 3     -19.274980  0.000004 -19.274976 -19.301762  2.565190e-06   
1.228600e-03 4     -19.274985  0.000005 -19.274980 -19.301765  3.592816e-06   
1.723400e-03 5     -19.274993  0.000008 -19.274986 -19.301770  5.067065e-06   
2.417400e-03 6     -19.275005  0.000011 -19.274994 -19.301777  7.181544e-06   
3.390900e-03 7     -19.275020  0.000015 -19.275005 -19.301787  1.021443e-05   
4.756500e-03 8     -19.275042  0.000022 -19.275020 -19.301801  1.456503e-05   
6.672000e-03 9     -19.275073  0.000031 -19.275041 -19.301819  2.080373e-05   
9.358900e-03 10    -19.275115  0.000045 -19.275070 -19.301846  2.975095e-05   
1.312780e-02 11    -19.275175  0.000064 -19.275111 -19.301883  4.257642e-05   
1.841450e-02 12    -19.275258  0.000091 -19.275167 -19.301934  6.096199e-05   
2.583020e-02 13    -19.275374  0.000130 -19.275244 -19.302005  8.730214e-05   
3.623230e-02 14    -19.275538  0.000185 -19.275353 -19.302104  1.250269e-04   
5.082340e-02 15    -19.275768  0.000262 -19.275506 -19.302243  1.790943e-04   
7.129060e-02 16    -19.276089  0.000377 -19.275712 -19.302433  2.601509e-04   
1.000000e-01 17    -19.276516  0.000567 -19.275949 -19.302678  3.961844e-04   
1.402710e-01 18    -19.277051  0.000904 -19.276147 -19.302972  6.403124e-04   
1.967600e-01 19    -19.277688  0.001495 -19.276193 -19.303292  1.076599e-03   
2.759970e-01 20    -19.278384  0.002519 -19.275865 -19.303581  1.848801e-03   
3.871000e-01 21    -19.279010  0.004262 -19.274748 -19.303703  3.201083e-03   
5.431000e-01 22    -19.279266  0.007188 -19.272078 -19.303379  5.545284e-03   
7.617000e-01 23    -19.278482  0.011997 -19.266485 -19.302062  9.542092e-03   
1.068500e+00 24    -19.275686  0.019352 -19.256334 -19.299015  1.595398e-02   
1.498800e+00 25    -19.270967  0.028355 -19.242612 -19.294309  2.452125e-02   
2.102400e+00 26    -19.265523  0.036929 -19.228594 -19.288835  3.393518e-02   
2.949000e+00 27    -19.259909  0.044020 -19.215889 -19.283035  4.312700e-02   
4.136600e+00 28    -19.254393  0.049291 -19.205102 -19.277145  5.106184e-02   
5.802500e+00 29    -19.249190  0.052935 -19.196255 -19.271370  5.677517e-02   
8.139200e+00 30    -19.244392  0.055681 -19.188711 -19.265810  5.988318e-02   
1.141700e+01 31    -19.240113  0.058403 -19.181709 -19.260610  6.090274e-02   
1.601470e+01 32    -19.236931  0.061756 -19.175175 -19.256391  6.129916e-02   
2.246400e+01 33    -19.236181  0.066378 -19.169803 -19.254512  6.325554e-02   
3.151050e+01 34    -19.239343  0.073627 -19.165716 -19.256617  6.955747e-02   
4.420010e+01 35    -19.248398  0.084981 -19.163416 -19.265159  8.216841e-02   
6.200000e+01 36    -19.268651  0.100783 -19.167868 -19.286426  1.011923e-01   
8.577500e+01 37    -19.309061  0.120669 -19.188392 -19.330734  1.256672e-01   
1.095500e+02 38    -19.362820  0.140886 -19.221934 -19.390831  1.501898e-01   
1.333250e+02 39    -19.428978  0.162466 -19.266512 -19.465775  1.757253e-01   
1.571000e+02 40    -19.510215  0.184220 -19.325995 -19.557656  2.012502e-01   
1.808750e+02 41    -19.609481  0.207792 -19.401689 -19.667633  2.279307e-01   
2.046500e+02 42    -19.723933  0.239651 -19.484283 -19.791553  2.613414e-01   
2.284250e+02 43    -19.847545  0.284553 -19.562992 -19.923633  3.062433e-01   
2.522000e+02 44    -19.978041  0.341343 -19.636699 -20.061583  3.625173e-01   
2.759750e+02 45    -20.114452  0.408748 -19.705705 -20.204473  4.292704e-01   
2.997500e+02 46    -20.255932  0.486163 -19.769769 -20.351461  5.058862e-01   
3.235250e+02 47    -20.401667  0.573241 -19.828426 -20.501579  5.918408e-01 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')